<a href="https://colab.research.google.com/github/Peheppy/CIFAR10-with-MLP/blob/main/CIFAR10_with_MLPs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIFAR10 with MLPs


## Instalação e Importações

In [5]:
%%capture
!pip install torchinfo
!mkdir -p logs

import torch
import torchvision
import torch.nn as nn
from torchvision.transforms import v2
from torchinfo import summary
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os
import json
import random
from tqdm import tqdm
from typing import Sequence, Callable
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

sns.set_style('darkgrid')

Configuração Inicial

In [6]:
# Configuração de dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo selecionado: {device}")

# Função para setar seed
def set_seed(seed):
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.use_deterministic_algorithms(False)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)

Dispositivo selecionado: cuda


## Dataset Setup

In [7]:
#@markdown ## This cell defines two data preprocessing pipelines for input images.
#@markdown ### The first pipeline includes data augmentation techniques such as random horizontal flipping, small rotations, and brightness/contrast adjustments to improve model generalization during training.
#@markdown ### The second pipeline is for evaluation (testing) and includes only resizing, conversion to tensors, and normalization, without any random transformations.
#@markdown ### Both pipelines ensure that the input format matches the model’s expectations.

# Transformações
transform = v2.Compose([
    v2.RandomHorizontalFlip(),
    v2.RandomRotation(10),
    v2.ColorJitter(brightness=0.2, contrast=0.2),
    v2.Resize((64,64)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.491, 0.482, 0.446), std=(0.247, 0.243, 0.261)),
])

test_transform = v2.Compose([
    v2.Resize((64, 64)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.491, 0.482, 0.446), std=(0.247, 0.243, 0.261)),
])

In [8]:
# Carregar datasets
full_train_ds = torchvision.datasets.CIFAR10(
    root=".", train=True, download=True, transform=transform)
test_ds = torchvision.datasets.CIFAR10(
    root=".", train=False, download=True, transform=test_transform)

100%|██████████| 170M/170M [00:13<00:00, 12.8MB/s]


###Splitting Data


####Theses cells split the CIFAR-10 training set into two subsets: 85% for training and 15% for validation. A fixed random seed ensures the split is reproducible. The test set remains unchanged with 10,000 images.

In [9]:
# Split treino/validação
prng = torch.Generator().manual_seed(42)
length = len(full_train_ds)
train_ds, val_ds = torch.utils.data.random_split(
    full_train_ds, [int(0.85*length), int(0.15*length)], generator=prng)

## Model definition (MLP)

In [10]:
class MLP(nn.Module):
    def __init__(
            self,
            input_size: int = 3*64*64,
            num_classes: int = 10,
            activation_function: Callable = nn.ReLU(),
            layer_sizes: Sequence[int] = [256, 512, 256],
            dropout_rate: float = 0.3
        ):
        super().__init__()

        sizes = [input_size, *layer_sizes, num_classes]
        layers = []

        for i in range(len(sizes)-1):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
            if i < len(layer_sizes):
                layers.append(nn.LayerNorm(sizes[i+1]))
                layers.append(activation_function)
                layers.append(nn.Dropout(dropout_rate))

        self.classifier = nn.Sequential(*layers)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x_flat = self.flatten(x)
        return self.classifier(x_flat)

# Classes Auxiliares

In [11]:
class AverageMeter:
    """Computa e armazena média e valor atual"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [12]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

# Training Utilities

In [13]:
def train_step(model, train_dl, loss_fn, optimizer, device):
    model.train()
    train_loss = AverageMeter()
    train_acc = AverageMeter()

    for x, y in tqdm(train_dl, desc="Training", leave=False):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(dim=1)
        acc = (preds == y).float().mean()
        train_loss.update(loss.item(), x.size(0))
        train_acc.update(acc.item(), x.size(0))

    return train_loss.avg, train_acc.avg

In [14]:
def eval_step(model, dl, loss_fn, device, name="Validation"):
    model.eval()
    loss_meter = AverageMeter()
    acc_meter = AverageMeter()

    with torch.no_grad():
        for x, y in tqdm(dl, desc=name, leave=False):
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = loss_fn(outputs, y)
            preds = outputs.argmax(dim=1)
            acc = (preds == y).float().mean()
            loss_meter.update(loss.item(), x.size(0))
            acc_meter.update(acc.item(), x.size(0))

    return loss_meter.avg, acc_meter.avg

# Configurações dos Experimentos

In [15]:
base_config = {
    "input_size": 3*64*64,
    "num_classes": 10,
    "n_epochs": 200,
    "bs_train": 256,
    "bs_val_test": 512,
    "loss_fn": nn.CrossEntropyLoss(),
    "patience": 15
}

In [16]:
model_configs = [
    {
        "name": "Baseline",
        "learning_rate": 4e-3,
        "optimizer": torch.optim.AdamW,
        "activation_fn": nn.LeakyReLU(),
        "layer_sizes": [256, 512, 256],
        "bs_train": 100,
        "dropout_rate": 0.3
    },
    {
        "name": "SmallNet",
        "learning_rate": 1e-3,
        "optimizer": torch.optim.Adam,
        "activation_fn": nn.ReLU(),
        "layer_sizes": [128, 256, 128],
        "bs_train": 200,
        "dropout_rate": 0.2
    },
    {
        "name": "DeepNet",
        "learning_rate": 3e-4,
        "optimizer": torch.optim.RMSprop,
        "activation_fn": nn.LeakyReLU(),
        "layer_sizes": [256, 512, 512, 256],
        "dropout_rate": 0.4
    }
]

In [17]:
# Combinar com configuração base
for config in model_configs:
    config.update(base_config)

 # Classe de Experimentos

In [18]:
class ModelExperiment:
    def __init__(self, config):
        self.config = config
        self.model = MLP(
            input_size=config["input_size"],
            num_classes=config["num_classes"],
            activation_function=config["activation_fn"],
            layer_sizes=config["layer_sizes"],
            dropout_rate=config["dropout_rate"]
        ).to(device)
        self.history = {
            "train_loss": [], "train_acc": [],
            "val_loss": [], "val_acc": [],
            "lr": []  # Adicionamos o histórico de learning rate
        }

    def run(self, train_dl, val_dl, test_dl=None):
        print(f"\n{'='*50}")
        print(f"Treinando modelo: {self.config['name']}")
        print(f"Configuração: {self.config}\n")

        optimizer = self.config["optimizer"](
            self.model.parameters(), lr=self.config["learning_rate"])

        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', patience=2)

        early_stop = EarlyStopping(patience=self.config["patience"])

        for epoch in range(self.config["n_epochs"]):
            train_loss, train_acc = train_step(
                self.model, train_dl, self.config["loss_fn"], optimizer, device)
            val_loss, val_acc = eval_step(
                self.model, val_dl, self.config["loss_fn"], device)

            # Registrar learning rate atual
            current_lr = optimizer.param_groups[0]['lr']

            # Atualizar histórico
            self.history["train_loss"].append(train_loss)
            self.history["train_acc"].append(train_acc)
            self.history["val_loss"].append(val_loss)
            self.history["val_acc"].append(val_acc)
            self.history["lr"].append(current_lr)

            # Atualizar scheduler e early stopping
            scheduler.step(val_loss)
            early_stop(val_loss)

            print(f"Epoch {epoch+1}/{self.config['n_epochs']} | "
                  f"LR: {current_lr:.2e} | "
                  f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
                  f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

            if early_stop.early_stop:
                print("Early stopping acionado!")
                break

        # Avaliação final
        if test_dl:
            test_loss, test_acc = eval_step(
                self.model, test_dl, self.config["loss_fn"], device, "Testing")
            print(f"\nResultados Finais - Loss: {test_loss:.4f} Acc: {test_acc:.4f}")
            self.history["test_loss"] = test_loss
            self.history["test_acc"] = test_acc

        return self

# Execução dos Experimentos

In [19]:
# Preparar DataLoaders
train_dl = torch.utils.data.DataLoader(
    train_ds, batch_size=base_config["bs_train"], shuffle=True)
val_dl = torch.utils.data.DataLoader(
    val_ds, batch_size=base_config["bs_val_test"], shuffle=False)
test_dl = torch.utils.data.DataLoader(
    test_ds, batch_size=base_config["bs_val_test"], shuffle=False)

# Executar experimentos
experiments = []
for config in model_configs:
    experiment = ModelExperiment(config).run(train_dl, val_dl, test_dl)
    experiments.append(experiment)


Treinando modelo: Baseline
Configuração: {'name': 'Baseline', 'learning_rate': 0.004, 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'activation_fn': LeakyReLU(negative_slope=0.01), 'layer_sizes': [256, 512, 256], 'bs_train': 256, 'dropout_rate': 0.3, 'input_size': 12288, 'num_classes': 10, 'n_epochs': 200, 'bs_val_test': 512, 'loss_fn': CrossEntropyLoss(), 'patience': 15}



Epoch 1/200 | LR: 4.00e-03 | Train Loss: 1.9322 Acc: 0.2966 | Val Loss: 1.7352 Acc: 0.3803


Epoch 2/200 | LR: 4.00e-03 | Train Loss: 1.7230 Acc: 0.3805 | Val Loss: 1.6584 Acc: 0.4068


Epoch 3/200 | LR: 4.00e-03 | Train Loss: 1.6429 Acc: 0.4141 | Val Loss: 1.5810 Acc: 0.4352


Epoch 4/200 | LR: 4.00e-03 | Train Loss: 1.6185 Acc: 0.4225 | Val Loss: 1.5301 Acc: 0.4492


...

Epoch 74/200 | LR: 3.00e-09 | Train Loss: 1.4689 Acc: 0.4803 | Val Loss: 1.3772 Acc: 0.5127


Epoch 75/200 | LR: 3.00e-09 | Train Loss: 1.4693 Acc: 0.4784 | Val Loss: 1.3829 Acc: 0.5105


Epoch 76/200 | LR: 3.00e-09 | Train Loss: 1.4651 Acc: 0.4836 | Val Loss: 1.3764 Acc: 0.5065


Epoch 77/200 | LR: 3.00e-09 | Train Loss: 1.4680 Acc: 0.4770 | Val Loss: 1.3808 Acc: 0.5084
Early stopping acionado!



Resultados Finais - Loss: 1.3837 Acc: 0.5088


# Visualização dos Resultados

In [20]:
# Função para plotar comparação
def plot_results(experiments, metric='loss'):
    plt.figure(figsize=(12, 6))
    for exp in experiments:
        plt.plot(exp.history[f'train_{metric}'], label=f"{exp.config['name']} - Train")
        plt.plot(exp.history[f'val_{metric}'], '--', label=f"{exp.config['name']} - Val")
    plt.title(f'Comparação de Modelos - {metric.capitalize()}')
    plt.xlabel('Época')
    plt.ylabel(metric.capitalize())
    plt.legend()
    plt.grid(True)
    plt.show()

In [21]:
def plot_learning_rates(experiments):
    plt.figure(figsize=(12, 6))
    for exp in experiments:
        plt.plot(exp.history['lr'], label=exp.config['name'])
    plt.title('Evolução do Learning Rate')
    plt.xlabel('Época')
    plt.ylabel('Learning Rate (escala log)')
    plt.yscale('log')
    plt.legend()
    plt.grid(True)
    plt.show()

In [22]:
# Tabela comparativa
results = []
for exp in experiments:
    res = {
        "Modelo": exp.config["name"],
        "Arquitetura": str(exp.config["layer_sizes"]),
        "Optimizer": exp.config["optimizer"].__name__,
        "LR Inicial": exp.config["learning_rate"],
        "LR Final": exp.history["lr"][-1],
        "Max Train Acc": f"{max(exp.history['train_acc'])*100:.2f}%",
        "Max Val Acc": f"{max(exp.history['val_acc'])*100:.2f}%",
        "Test Acc": f"{exp.history.get('test_acc', 0)*100:.2f}%",
        "Épocas": len(exp.history['train_loss'])
    }
    results.append(res)

results_df = pd.DataFrame(results)
print("\nResumo Comparativo dos Modelos:")
display(results_df.style.background_gradient(cmap='Blues'))


Resumo Comparativo dos Modelos:


,Modelo,Arquitetura,Optimizer,LR Inicial,LR Final,Max Train Acc,Max Val Acc,Test Acc,Épocas
0,Baseline,"[256, 512, 256]",AdamW,0.004000,0.000000,56.91%,57.00%,56.30%,46
1,SmallNet,"[128, 256, 128]",Adam,0.001000,0.000000,56.77%,57.55%,56.16%,77
2,DeepNet,"[256, 512, 512, 256]",RMSprop,0.000300,0.000000,48.36%,51.60%,50.88%,77


# Salvar Resultados

In [23]:
# Criar diretório para salvar resultados
!mkdir -p results

In [24]:
# Salvar resultados em CSV
results_df.to_csv("results/model_comparison.csv", index=False)

In [25]:
# Salvar modelos e históricos
for exp in experiments:
    model_name = exp.config["name"].replace(" ", "_").lower()
    # Salvar modelo
    torch.save(exp.model.state_dict(), f"results/{model_name}_model.pth")
    # Salvar histórico
    with open(f"results/{model_name}_history.json", 'w') as f:
        json.dump(exp.history, f)

print("Resultados salvos na pasta 'results'")

Resultados salvos na pasta 'results'
